# Example: Calculate Delta, Theta, Vega, Rho, and Gamma for Short and Long Options Contracts
[The Greeks](https://en.wikipedia.org/wiki/en:Greeks_(finance)?variant=zh-tw) are quantitative measures that describe the sensitivity of an American option's price to various risk factors. Understanding and monitoring the Greeks is crucial for effective options trading and risk management. The key Greeks for American options include `delta,` `gamma,` `vega,` `theta,` and `rho,` each quantifying how the option's value changes with movements in the underlying asset price, volatility, time decay, and interest rates.

## Learning objectives
This example will familiarize you with calculating [the Greeks](https://en.wikipedia.org/wiki/en:Greeks_(finance)?variant=zh-tw) for long and short American-style option contracts, and give you a chance to test your calculations using actual contracts from the [NVIDIA Corporation](https://finance.yahoo.com/quote/NVDA/) (ticker `NVDA`). Let's get started!
* __Prerequisites__: We'll begin by setting some parameter values for the contracts, then we'll build contract models for the example call and put options on `NVDA.`
* __Task 1__: Compute the Greeks for a Long `NVDA` Call Option.
    * `TODO`: Compute `delta` and check the computed and observed `delta` values for a long call
    * `TODO`: Compute `theta` and check the computed and observed `theta` values for a long call
    * `TODO`: Compute `vega` and check the computed and observed `vega` values for a long call
    * `TODO`: Compute `rho` and check the computed and observed `rho` values for a long call
    * `TODO`: Compute `gamma` and check the computed and observed `gamma` values for a long call
* __Task 2__: Compute the Greeks for a Long `NVDA` Put Option.
    * `TODO`: Compute `delta` and check the computed and observed `delta` values for a long put
    * `TODO`: Compute `theta` and check the computed and observed `theta` values for a long put
    * `TODO`: Compute `vega` and check the computed and observed `vega` values for a long put
    * `TODO`: Compute `rho` and check the computed and observed `rho` values for a long put
    * `TODO`: Compute `gamma` and check the computed and observed `gamma` values for a long put
* __Task 3__: Conversion of the Greeks for Long Contracts to Short Contracts

## Setup
Set up the computational environment by including the `Include.jl` file. The `Include.jl` file loads external packages, various functions we will use in the exercise, and custom types to model the components of our example problem.
* For additional information on functions and types used in this material, see the [Julia programming language documentation](https://docs.julialang.org/en/v1/) and the [VLQuantitativeFinancePackage.jl documentation](https://github.com/varnerlab/VLQuantitativeFinancePackage.jl). 

In [3]:
include("Include.jl")

    Updating git-repo `https://github.com/varnerlab/VLQuantitativeFinancePackage.jl.git`
    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
   Installed SentinelArrays ─ v1.4.7
   Installed Adapt ────────── v4.1.1
   Installed Distances ────── v0.10.12
   Installed ForwardDiff ──── v0.10.37
    Updating `~/Desktop/julia_work/CHEME-5660-CourseRepository-Fall-2024/lectures/week-11/L11a/Project.toml`
  [336ed68f] + CSV v0.10.15
⌅ [5ae59095] + Colors v0.12.11
  [a93c6f00] + DataFrames v1.7.0
  [31c24e10] + Distributions v0.25.112
  [5789e2e9] + FileIO v1.16.4
  [09f84164] + HypothesisTests v0.11.3
  [033835bb] + JLD2 v0.5.7
  [91a5bcdd] + Plots v1.40.8
  [08abe8d2] + PrettyTables v2.4.0
  [10745b16] + Statistics v1.11.1
  [f3b207a7] + StatsPlots v0.15.7
  [4119e0bf] + VLQuantitativeFinancePackage v1.0.0-DEV `https://github.com/varnerlab/VLQuantitativeFinancePackage.jl.git#main`
  [ade2ca70] ~ Dates ⇒ v1.11.0
  [37e2e46d] ~ LinearAlgebra ⇒ v1.11.0
 

## Prerequisites
Let's build models for options on [NVIDIA Corporation](https://finance.yahoo.com/quote/NVDA/) as the underlying asset with ticker `NVDA,` in particular, out-of-the-money (OTM) short call and put options.  First, we set some constants and parameter values that will be used in the notebook, and then we'll build contract models.

### Parameters and constants
Let's set a value for the step size `Δt,` i.e., `1 day` in units of years (assuming a `365-day` year), the risk-free rate at the time we gathered the data for the option contracts; we approximate this value using the [yield on the 10-year Treasury Note](https://ycharts.com/indicators/10_year_treasury_rate). We also set a value for the underlying share price `Sₒ` and the number of levels in the binomial tree model, `h`; we assume 6-levels per day.

In [6]:
Δt = (1/365);
risk_free_rate = 0.0431; # yield 10-year treasury note on 13-Jun-2024
Sₒ = 131.88; # after close on 14-Jun-2024 (close: 131.88)
h = 366; # 6-points per trading day

### Contract models
Next, we construct contract models for the options in this example. We begin by building an instance of the [MyAmericanCallContractModel type](https://varnerlab.github.io/VLQuantitativeFinancePackage.jl/dev/derivatives/#VLQuantitativeFinancePackage.MyAmericanCallContractModel) using the [custom build(...) method](https://varnerlab.github.io/VLQuantitativeFinancePackage.jl/dev/derivatives/#VLQuantitativeFinancePackage.build-Tuple{Type{MyAmericanCallContractModel},%20NamedTuple}). The [build(...) method](https://varnerlab.github.io/VLQuantitativeFinancePackage.jl/dev/derivatives/#VLQuantitativeFinancePackage.build-Tuple{Type{MyAmericanCallContractModel},%20NamedTuple}) takes two arguments: the type to build, and a [NamedTuple](https://docs.julialang.org/en/v1/base/base/#Core.NamedTuple) containing the strike price `K`, the `premium` paid for the option, the days to expiration `DTE` (units: years), the `sense = 1` (long), the number of contracts in the `copy` field and the implied volatility in the `IV` field.

We save the call option contract model in the `call_option_model` variable:

In [8]:
call_option_model = build(MyAmericanCallContractModel, (
    K = 136.0, 
    premium = 9.23, # mark price
    DTE = (61.0)*Δt,
    sense = 1,
    copy = 1,
    IV = 0.4848
));

The put contract, which is an instance of the [MyAmericanPutContractModel type](https://varnerlab.github.io/VLQuantitativeFinancePackage.jl/dev/derivatives/#VLQuantitativeFinancePackage.MyAmericanPutContractModel), is constructed in a similar way to the call option. We save the put option contract model in the `put_option_model` variable:

In [10]:
put_option_model = build(MyAmericanPutContractModel, (
    K = 128.0, 
    premium = 7.78, # mark price
    DTE = (61.0)*Δt,
    sense = 1,
    copy = 1,
    IV = 0.4787
));

__Note__: In both cases, we construct long contract models, i.e., we look at option dynamics from the perspective of a contract buyer. To convert to a short perspective, i.e., from the perspective of the contract seller, we multiply the Greek values by `-1`; see below.

## Task 1: Compute the Greeks for a Long Call Option
Let's begin our examination of the Greeks by calculating the Greek values for a long call option. From the buyer's viewpoint, a long call option is a substitute for owning shares. The buyer speculates that the underlying share price will go up during the contract and hopes to profit from the increase in the option contract's price. 
* However, the option contract price is a function of many different market factors, not just the share price of the underlying asset. The Greeks provide a quantitative means of computing the contract price's sensitivity to changes in market conditions.

Let's start our exploration of the Greeks by computing `delta` for the long call.

### Delta long call
The `delta` for an American option contract quantifies the rate of change in the option's premium $\mathcal{P}$ with respect to a `+1 USD/share` change in the underlying asset's price and is defined as:
$$
\Delta(\star) \equiv \frac{\partial\mathcal{P}}{\partial{S}}\,\Biggr|_{\star}
$$
where $\Delta(\star)$ is evaluated at the current state of the market $\star$, e.g., the current value of the underlying share price, the implied volatility of the contract, the number of days left until contract expiration, the current risk-free rate, etc. The quantity of $\Delta$ is bounded in the range $-1\leq\Delta\leq{1}$ and changes with various market conditions, i.e., $\Delta$ is _not_ constant.

* We estimate $\Delta$ using the [delta(...) function](https://varnerlab.github.io/VLQuantitativeFinancePackage.jl/dev/derivatives/#VLQuantitativeFinancePackage.delta). The [delta(...) function](https://varnerlab.github.io/VLQuantitativeFinancePackage.jl/dev/derivatives/#VLQuantitativeFinancePackage.delta) takes several arguments: the `call_option_model` instance, and several parameters used to compute the risk-neutral binomial share price tree, e.g., the number of tree levels `h,` the current share price  `Sₒ,` the number of days to expiration `T,` etc. and returns a value for `delta.` We store the estimated value for delta in the `delta_long_call` variable

In [14]:
delta_long_call = delta(call_option_model, h = h,  T = call_option_model.DTE, 
    σ = call_option_model.IV, Sₒ = Sₒ, μ = risk_free_rate)

0.4920000000000009

__Summary__: For a long call, $\Delta(\star)\geq{0}$ implies that the option premium $\mathcal{P}$ increases with increasing share price. This underscores the use case of long-call contracts as share replacements for underlying shares. 
* Further, $\Delta(\star)$ can be used to compute the equivalent number of shares that the option represents, denoted as $n_{c}$ shares per contract:
$$
n_{c} = 100\cdot\Delta_{c}(\star)
$$
where $\Delta_{c}(\star)$ denotes the $\Delta(\star)$ value for a long-call contract. Thus, this long call contract is equivalent to:

In [16]:
println("Number of share equivalents: $(delta_long_call*100) shares per contract")

Number of share equivalents: 49.20000000000009 shares per contract


#### Check: How good is our delta estimate?
The delta value for the `long-call` is `delta = 0.4979.` Let's use the [Julia @assert macro](https://docs.julialang.org/en/v1/base/base/#Base.@assert) in combination with the [Julia isapprox(...) function](https://docs.julialang.org/en/v1/base/math/#Base.isapprox) to compare the computed and observed values. 
* The [isapprox(...) function](https://docs.julialang.org/en/v1/base/math/#Base.isapprox) compares two values and returns `true` if the two values are equal to some absolute or relative tolerance, in this case, specified by the absolute tolerance `atol` variable. If `false` (values are not equal), an [AssertionError](https://docs.julialang.org/en/v1/base/base/#Core.AssertionError) is generated:

In [18]:
observed_value_delta_long_call = 0.4979; # NVDA 61-DTE K = 136 long call after close 14-Jun-2024 
@assert isapprox(observed_value_delta_long_call, delta_long_call, atol=1e-2) # good to second decimal place, and sign is correct!

### Theta long call
`Theta` measures the rate of change in the options premium $\mathcal{P}$ for a `1-day` decrease in the time to maturity `T` of the contract. The `theta` metric  is defined as:
$$
\begin{equation}
\Theta(\star) \equiv \frac{\partial\mathcal{P}}{\partial{T}}\,\Biggr|_{\star}
\end{equation}
$$
where $\Theta(\star)$ is evaluated at the current state of the market $\star$, e.g., the current value of the underlying share price, the implied volatility of the contract, the number of days left until contract expiration, the current risk-free rate, etc. 
* We estimate $\Theta$ using the [theta(...) function](https://varnerlab.github.io/VLQuantitativeFinancePackage.jl/dev/derivatives/#VLQuantitativeFinancePackage.theta). The [theta(...) function](https://varnerlab.github.io/VLQuantitativeFinancePackage.jl/dev/derivatives/#VLQuantitativeFinancePackage.theta) takes several arguments: the `call_option_model` instance, and several parameters used to compute the risk-neutral binomial share price tree, e.g., the number of tree levels `h,` the current share price  `Sₒ,` the number of days to expiration `T,` etc. and returns a value for `theta.` We store the estimated value for theta in the `theta_long_call` variable

In [20]:
theta_long_call = theta(call_option_model, h = h,  T = call_option_model.DTE, 
    σ = call_option_model.IV, Sₒ = Sₒ, μ = risk_free_rate)

-0.09199999999999875

__Summary__: For a long-call, $\Theta(\star)<0$ implies that the option premium $\mathcal{P}$ decreases as we approach the expiration of the contract. Thus, all else being constant, a long call will _lose value_ with time. This loss is referred to as the time-decay of the long call contract. For the `NVDA` call contract, we have a time-decay value of (units: USD per day per contract):

In [22]:
println("The long-call changes in value: $(theta_long_call*100) USD per day per contract")

The long-call changes in value: -9.199999999999875 USD per day per contract


#### Check: How good is our theta estimate?
This option's observed `theta` value is `theta = -0.0937` (long call). Let's use the [Julia @assert macro](https://docs.julialang.org/en/v1/base/base/#Base.@assert) in combination with the [Julia isapprox(...) function](https://docs.julialang.org/en/v1/base/math/#Base.isapprox) to compare the computed and observed values. 
* The [isapprox(...) function](https://docs.julialang.org/en/v1/base/math/#Base.isapprox) compares two values and returns `true` if the two values are equal to some absolute or relative tolerance, in this case, specified by the absolute tolerance `atol` variable. If `false` (values are not equal), an [AssertionError](https://docs.julialang.org/en/v1/base/base/#Core.AssertionError) is generated:

In [24]:
observed_value_theta_long_call = -0.0937; # NVDA 61-DTE K = 136 long call after close 14-Jun-2024 
@assert isapprox(observed_value_theta_long_call, theta_long_call, atol=1e-2) # good to second decimal place, and sign is correct!

### Vega long call
`Vega` measures the rate of change in the option's premium $\mathcal{P}$ with respect to a `+1%` change in the underlying asset's implied volatility `IV.` The `vega` metric is defined as:
$$
V(\star) \equiv \frac{\partial\mathcal{P}}{\partial{\sigma}}\Biggr|_{\star}
$$
where $\sigma$ denotes the implied volatility `IV.`  The `vega` is evaluated at the current state of the market $\star$, e.g., the current value of the underlying share price, the implied volatility of the contract, the number of days left until contract expiration, the current risk-free rate, etc. 
* We estimate `vega` using the [vega(...) function](https://varnerlab.github.io/VLQuantitativeFinancePackage.jl/dev/derivatives/#VLQuantitativeFinancePackage.vega). The [vega(...) function](https://varnerlab.github.io/VLQuantitativeFinancePackage.jl/dev/derivatives/#VLQuantitativeFinancePackage.vega) takes several arguments: the `call_option_model` instance, and several parameters used to compute the risk-neutral binomial share price tree, e.g., the number of tree levels `h,` the current share price  `Sₒ,` the number of days to expiration `T,` etc. and returns a value for `vega.` We store the estimated value for vega in the `vega_long_call` variable

In [26]:
vega_long_call = vega(call_option_model, h = h,  T = call_option_model.DTE, 
    σ = call_option_model.IV, Sₒ = Sₒ, μ = risk_free_rate)

0.21600000000000108

__Summary__: For the long call contract, $V(\star)>0$. Thus, the option premium for this long call will increase with increasing implied volatility. In other words, as the uncertainty in the future share price increases (measured by the implied volatility), an option contract becomes more expensive. This makes sense as the option seller demands additional compensation for the additional risk. 

#### Check: How good is our vega estimate?
This `vega` value for the `long-call` is `vega = 0.2159.` Let's use the [Julia @assert macro](https://docs.julialang.org/en/v1/base/base/#Base.@assert) in combination with the [Julia isapprox(...) function](https://docs.julialang.org/en/v1/base/math/#Base.isapprox) to compare the computed and observed values. 
* The [isapprox(...) function](https://docs.julialang.org/en/v1/base/math/#Base.isapprox) compares two values and returns `true` if the two values are equal to some absolute or relative tolerance, in this case, specified by the absolute tolerance `atol` variable. If `false` (values are not equal), an [AssertionError](https://docs.julialang.org/en/v1/base/base/#Core.AssertionError) is generated:

In [29]:
observed_value_vega_long_call = 0.2159; # NVDA 61-DTE K = 136 long call after close 14-Jun-2024 
@assert isapprox(observed_value_vega_long_call, vega_long_call, atol=1e-2) # good to the second decimal place, and sign is correct!

### Rho long call
`Rho` measures the rate of change in the options premium $\mathcal{P}$ for a `+1%` change in the risk-free rate $r_{f}$. The `rho` metric is defined as:
$$
\begin{equation}
\rho(\star)\equiv\frac{\partial\mathcal{P}}{\partial{r_{f}}}\Biggr|_{\star}
\end{equation}
$$
where $\rho(\star)$ is evaluated at the current state of the market $\star$, e.g., the current value of the underlying share price, the implied volatility of the contract, the number of days left until contract expiration, the current risk-free rate, etc. 
* We estimate $\rho$ using the [rho(...) function](https://varnerlab.github.io/VLQuantitativeFinancePackage.jl/dev/derivatives/#VLQuantitativeFinancePackage.rho). The [rho(...) function](https://varnerlab.github.io/VLQuantitativeFinancePackage.jl/dev/derivatives/#VLQuantitativeFinancePackage.rho) takes several arguments: the `call_option_model` instance, and several parameters used to compute the risk-neutral binomial share price tree, e.g., the number of tree levels `h,` the current share price  `Sₒ,` the number of days to expiration `T,` etc. and returns a value for `rho.` We store the estimated value for rho in the `rho_long_call` variable

In [31]:
rho_long_call = rho(call_option_model, h = h,  T = call_option_model.DTE, 
    σ = call_option_model.IV, Sₒ = Sₒ, μ = risk_free_rate)

0.0940000000000012

__Summary__: `rho` quantifies how much an option's price will theoretically change due to interest rate fluctuations, which is an important consideration, especially for longer-term options positions. In normal circumstances, the risk-free rate doesn't typically change. However, when interest rates increase during inflationary periods, `rho` can influence option premiums. For this option, changes in $r_{f}$ results in:

In [33]:
println("Change in contract value by a 1% increase in risk-free rate: $(rho_long_call*100) USD per contract")

Change in contract value by a 1% increase in risk-free rate: 9.40000000000012 USD per contract


#### Check: How good is our rho estimate?
The `rho` value for the `long-call` is `rho = 0.0950.` Let's use the [Julia @assert macro](https://docs.julialang.org/en/v1/base/base/#Base.@assert) in combination with the [Julia isapprox(...) function](https://docs.julialang.org/en/v1/base/math/#Base.isapprox) to compare the computed and observed values. 
* The [isapprox(...) function](https://docs.julialang.org/en/v1/base/math/#Base.isapprox) compares two values and returns `true` if the two values are equal to some absolute or relative tolerance, in this case, specified by the absolute tolerance `atol` variable. If `false` (values are not equal), an [AssertionError](https://docs.julialang.org/en/v1/base/base/#Core.AssertionError) is generated:

In [35]:
observed_value_rho_long_call = 0.0950; # NVDA 61-DTE K = 136 long call after close 14-Jun-2024 
@assert isapprox(observed_value_rho_long_call, rho_long_call, atol=1e-2) # good to the second decimal place, and sign is correct!

### Gamma long call
Finally, $\Gamma$ is the last of the common Greeks. The $\Gamma$ metric measures the rate of change in the `delta` for a `+1 USD/share` change in the underlying price and is defined as:
$$
\begin{equation}
\Gamma(\star) \equiv \frac{\partial^2\mathcal{P}}{\partial{S}^2}\Biggr|_{\star}
\end{equation}
$$
where $\Gamma(\star)$ is evaluated at the current state of the market $\star$, e.g., the current value of the underlying share price, the implied volatility of the contract, the number of days left until contract expiration, the current risk-free rate, etc. 
* We estimate $\Gamma$ using the [gamma(...) function](https://varnerlab.github.io/VLQuantitativeFinancePackage.jl/dev/derivatives/#VLQuantitativeFinancePackage.gamma). The [gamma(...) function](https://varnerlab.github.io/VLQuantitativeFinancePackage.jl/dev/derivatives/#VLQuantitativeFinancePackage.gamma) takes several arguments: the `call_option_model` instance, and several parameters used to compute the risk-neutral binomial share price tree, e.g., the number of tree levels `h,` the current share price  `Sₒ,` the number of days to expiration `T,` etc. and returns a value for `Gamma.` We store the estimated value for gamma in the `gamma_long_call` variable

In [37]:
gamma_long_call = gamma(call_option_model, h = h,  T = call_option_model.DTE, 
    σ = call_option_model.IV, Sₒ = Sₒ, μ = risk_free_rate)

0.022999999999999687

__Summary__: The long call option has a $\Gamma(\star)>0$, the premium's sensitivity to share price movements increases. This means that the premium becomes more responsive to changes in share price. The gamma value indicates how sensitive an option's delta is to changes in the underlying price, with higher gamma values indicating greater delta sensitivity.

#### Check: How good is our gamma estimate?
The `gamma` value for the `long-call` is `gamma = 0.0152.` Let's use the [Julia @assert macro](https://docs.julialang.org/en/v1/base/base/#Base.@assert) in combination with the [Julia isapprox(...) function](https://docs.julialang.org/en/v1/base/math/#Base.isapprox) to compare the computed and observed values. 
* The [isapprox(...) function](https://docs.julialang.org/en/v1/base/math/#Base.isapprox) compares two values and returns `true` if the two values are equal to some absolute or relative tolerance, in this case, specified by the absolute tolerance `atol` variable. If `false` (values are not equal), an [AssertionError](https://docs.julialang.org/en/v1/base/base/#Core.AssertionError) is generated:

In [40]:
observed_value_gamma_long_call = 0.0152; # NVDA 61-DTE K = 136 long call after close 14-Jun-2024 
@assert isapprox(observed_value_gamma_long_call, gamma_long_call, atol=1e-2) # good to the second decimal place, and sign is correct!

## Task 2: Compute the Greeks for a Long Put Option
Now, let's look at the Greek values for a long-put option. In each case, we'll compare the computed value with values observed in the market and contrast the results of the long put contract with those of the long call. Let's begin with `delta.`

### Delta long put
Like the call option, we estimate $\Delta$ using the [delta(...) function](https://varnerlab.github.io/VLQuantitativeFinancePackage.jl/dev/derivatives/#VLQuantitativeFinancePackage.delta), but in this case using the `put_option_model` instance, all other parameters held the same:

In [43]:
delta_long_put = delta(put_option_model, h = h,  T = put_option_model.DTE, 
    σ = put_option_model.IV, Sₒ = Sₒ, μ = risk_free_rate)

-0.3879999999999999

__Summary__: The $\Delta(\star)<0$ for the long-put (opposite sign of the call). Thus, as the underlying share price increases, the value of the put option _decreases_. This is consistent with the notion that long puts make a `bearish` directional assumption, i.e., a buyer will purchase a put option if they believe the underlying asset's share price will decrease. 
* _What about the number of share equivalents?_ The number of share equivalents for a long put, denoted as $n_{p}$, will be _negative_. Huh, what? This means that we are _short_ the shares but _long_ the put (confused yet?). By being _short the shares_, we benefit from the decrease in the underlying price. For this long put, we have a $n_{p}$ defined as:
$$
n_{p} = 100\cdot\Delta_{p}(\star)
$$
where $\Delta_{p}(\star)$ denotes the $\Delta(\star)$ value for a long-put contract. Thus, this long put contract is equivalent to:

In [45]:
println("Number of share equivalents for our long-put: $(delta_long_put*100) shares per contract")

Number of share equivalents for our long-put: -38.79999999999999 shares per contract


#### Check: How good is our delta estimate?
The observed `delta` value for the `long put` is `delta = -0.3847` (long put). Let's use the [Julia @assert macro](https://docs.julialang.org/en/v1/base/base/#Base.@assert) in combination with the [Julia isapprox(...) function](https://docs.julialang.org/en/v1/base/math/#Base.isapprox) to compare the computed and observed values. 
* The [isapprox(...) function](https://docs.julialang.org/en/v1/base/math/#Base.isapprox) compares two values and returns `true` if the two values are equal to some absolute or relative tolerance, in this case, specified by the absolute tolerance `atol` variable. If `false` (values are not equal), an [AssertionError](https://docs.julialang.org/en/v1/base/base/#Core.AssertionError) is generated:

In [47]:
observed_value_delta_long_put = -0.3847; # NVDA 61-DTE K = 128 long put after close 14-Jun-2024 
@assert isapprox(observed_value_delta_long_put, delta_long_put, atol=1e-2) # good to second decimal place, and sign is correct!

### Theta
Like the call option, we estimate $\Theta$ using the [theta(...) function](https://varnerlab.github.io/VLQuantitativeFinancePackage.jl/dev/derivatives/#VLQuantitativeFinancePackage.theta), but in this case using the `put_option_model` instance, all other parameters held the same:

In [49]:
theta_long_put = theta(put_option_model, h = h,  T = put_option_model.DTE, 
    σ = put_option_model.IV, Sₒ = Sₒ, μ = risk_free_rate)

-0.07499999999999929

__Summary__: The $\Theta(\star)<0$ for the long-put (same sign as the call option). Thus, the long-call and the long-put behave similarly in that the price of the option contract _decreases_ as the contract approaches expiration, where all else is held constant. For the `NVDA` put contract, we have a time-decay value of (units: USD per day per contract):

In [51]:
println("The long-put changes in value: $(theta_long_put*100) USD per day per contract")

The long-put changes in value: -7.499999999999929 USD per day per contract


#### Check: How good is our theta estimate?
The observed `theta` value for the `long put` is `theta = -0.0726` (long put). Let's use the [Julia @assert macro](https://docs.julialang.org/en/v1/base/base/#Base.@assert) in combination with the [Julia isapprox(...) function](https://docs.julialang.org/en/v1/base/math/#Base.isapprox) to compare the computed and observed values. 
* The [isapprox(...) function](https://docs.julialang.org/en/v1/base/math/#Base.isapprox) compares two values and returns `true` if the two values are equal to some absolute or relative tolerance, in this case, specified by the absolute tolerance `atol` variable. If `false` (values are not equal), an [AssertionError](https://docs.julialang.org/en/v1/base/base/#Core.AssertionError) is generated:

In [53]:
observed_value_theta_long_put = -0.0726; # NVDA 61-DTE K = 128 long put after close 14-Jun-2024 
@assert isapprox(observed_value_theta_long_put, theta_long_put, atol=1e-2) # good to the second decimal place, and sign is correct!

### Vega
Like the call option, we estimate `Vega` using the [vega(...) function](https://varnerlab.github.io/VLQuantitativeFinancePackage.jl/dev/derivatives/#VLQuantitativeFinancePackage.vega), but in this case using the `put_option_model` instance, all other parameters held the same:

In [55]:
vega_long_put = vega(put_option_model, h = h,  T = put_option_model.DTE, 
    σ = put_option_model.IV, Sₒ = Sₒ, μ = risk_free_rate)

0.20700000000000074

__Summary__: For the long put contract, $V(\star)>0$. Thus, the option premium for the long put (like the long call) will increase with implied volatility; as the uncertainty in the future share price increases (measured by the implied volatility), an option contract becomes more expensive (both call and puts). This makes sense as the option seller will demand additional compensation for the additional risk. 

#### Check: How good is our vega estimate?
The observed `vega` value for the `long put` is `vega = 0.2068.` Let's use the [Julia @assert macro](https://docs.julialang.org/en/v1/base/base/#Base.@assert) in combination with the [Julia isapprox(...) function](https://docs.julialang.org/en/v1/base/math/#Base.isapprox) to compare the computed and observed values. 
* The [isapprox(...) function](https://docs.julialang.org/en/v1/base/math/#Base.isapprox) compares two values and returns `true` if the two values are equal to some absolute or relative tolerance, in this case, specified by the absolute tolerance `atol` variable. If `false` (values are not equal), an [AssertionError](https://docs.julialang.org/en/v1/base/base/#Core.AssertionError) is generated:

In [58]:
observed_value_vega_long_put = 0.2068; # NVDA 61-DTE K = 128 long put after close 14-Jun-2024 
@assert isapprox(observed_value_vega_long_put, vega_long_put, atol=1e-2) # good to the second decimal place, and sign is correct!

### Rho
Like the call option, we estimate $\rho$ using the [rho(...) function](https://varnerlab.github.io/VLQuantitativeFinancePackage.jl/dev/derivatives/#VLQuantitativeFinancePackage.rho), but in this case using the `put_option_model` instance, all other parameters held the same:

In [60]:
rho_long_put = rho(put_option_model, h = h,  T = put_option_model.DTE, 
    σ = put_option_model.IV, Sₒ = Sₒ, μ = risk_free_rate)

-0.0829999999999993

__Summary__: `rho` quantifies how much an option's price will theoretically change due to interest rate fluctuations, which is an important consideration, especially for longer-term options positions. However, unlike the long call contract, $\rho(\star)<0$ for the long-put contract.

In [62]:
println("Change in put contract price by a 1% increase in risk-free rate: $(rho_long_put*100) USD per contract")

Change in put contract price by a 1% increase in risk-free rate: -8.29999999999993 USD per contract


#### Check: How good is our rho estimate?
The observed `rho` value for the `long put` is `rho = -0.0902` Let's use the [Julia @assert macro](https://docs.julialang.org/en/v1/base/base/#Base.@assert) in combination with the [Julia isapprox(...) function](https://docs.julialang.org/en/v1/base/math/#Base.isapprox) to compare the computed and observed values. 
* The [isapprox(...) function](https://docs.julialang.org/en/v1/base/math/#Base.isapprox) compares two values and returns `true` if the two values are equal to some absolute or relative tolerance, in this case, specified by the absolute tolerance `atol` variable. If `false` (values are not equal), an [AssertionError](https://docs.julialang.org/en/v1/base/base/#Core.AssertionError) is generated:

In [64]:
observed_value_rho_long_put = -0.0902; # NVDA 61-DTE K = 128 long put after close 14-Jun-2024 
@assert isapprox(observed_value_rho_long_put, rho_long_put, atol=1e-2) # good to the second decimal place, and sign is correct!

### Gamma
Finally, like the $\Gamma$ calculation for the long-call, we estimate $\Gamma$ using the [gamma(...) function](https://varnerlab.github.io/VLQuantitativeFinancePackage.jl/dev/derivatives/#VLQuantitativeFinancePackage.gamma), but in this case using the `put_option_model` instance, all other parameters held the same:

In [66]:
gamma_long_put = gamma(put_option_model, h = h,  T = put_option_model.DTE, 
    σ = put_option_model.IV, Sₒ = Sₒ, μ = risk_free_rate)

0.020000000000000462

__Summary__: The long put option has a $\Gamma(star)>0$, which increases the premium's sensitivity to share price movements, i.e., the premium becomes more responsive to changes in share price. The $\Gamma$ value indicates how sensitive an option's `delta` is to changes in the underlying price, with higher $\Gamma$ values indicating greater `delta` sensitivity.

#### Check: How good is our gamma estimate?
The observed `gamma` value for the `long put` is `gamma = 0.0150.` Let's use the [Julia @assert macro](https://docs.julialang.org/en/v1/base/base/#Base.@assert) in combination with the [Julia isapprox(...) function](https://docs.julialang.org/en/v1/base/math/#Base.isapprox) to compare the computed and observed values. 
* The [isapprox(...) function](https://docs.julialang.org/en/v1/base/math/#Base.isapprox) compares two values and returns `true` if the two values are equal to some absolute or relative tolerance, in this case, specified by the absolute tolerance `atol` variable. If `false` (values are not equal), an [AssertionError](https://docs.julialang.org/en/v1/base/base/#Core.AssertionError) is generated:

In [69]:
observed_value_gamma_long_put = 0.0150; # NVDA 61-DTE K = 128 long put after close 14-Jun-2024 
@assert isapprox(observed_value_gamma_long_put, gamma_long_put, atol=1e-2) # good to the second decimal place, and sign is correct!

## Task 3: Conversion of the Greeks to Short Contracts
We computed `delta, theta, vega, rho` and `gamma` values for `long` put and call contacts in the previous tasks. The immediate question is, then, what about short contracts? Computing the Greek values for short contracts is easy: the Greek values for long and short Greek contracts must sum to `0`:

* Denote the short value of a Greek by an overbar, i.e., the short `delta` for a call evaluated at market conditions $\star$ would be given by $\bar{\Delta}_{c}(\star)$. Then, the following condition must hold (for all market states $\star$):
$$
\begin{equation}
\bar{\Delta}_{c}(\star) + \Delta_{c}\left(\star\right) = 0\quad\forall\star
\end{equation}
$$

It's important to note that the interconversion between long and short Greek values is a universal concept applicable to all the Greeks we have explored. This understanding will serve as a solid foundation for your options trading knowledge.

`Unhide` the code block below to see how we constructed a table for the Greeks for the long-call `NVDA` contract.

In [ ]:
let
    
    sense = -1;
    greek_dataframe = DataFrame();

    # delta -
    delta_row = (
        greek = "delta", 
        call_long_calc = delta_long_call,
        call_short_calc = (sense)*delta_long_call,
        check = delta_long_call + (sense)*delta_long_call,
        call_long_obs = observed_value_delta_long_call,
        error_percentage = ((delta_long_call - observed_value_delta_long_call)/(observed_value_delta_long_call))*100
    )
    push!(greek_dataframe,delta_row);

    # theta
    theta_row = (
        greek = "theta", 
        call_long_calc = theta_long_call,
        call_short_calc = (sense)*theta_long_call,
        check = theta_long_call + (sense)*theta_long_call,
        call_long_obs = observed_value_theta_long_call,
        error_percentage = ((theta_long_call - observed_value_theta_long_call)/(observed_value_theta_long_call))*100,
    )
    push!(greek_dataframe,theta_row);

    # vega -
    vega_row = (
        greek = "vega", 
        call_long_calc = vega_long_call,
        call_short_calc = (sense)*vega_long_call,
        check = vega_long_call + (sense)*vega_long_call,
        call_long_obs = observed_value_vega_long_call,
        error_percentage = ((vega_long_call - observed_value_vega_long_call)/(observed_value_vega_long_call))*100
    )
    push!(greek_dataframe,vega_row);

    # rho -
    rho_row = (
        greek = "rho", 
        call_long_calc = rho_long_call,
        call_short_calc = (sense)*rho_long_call,
        check = rho_long_call + (sense)*rho_long_call,
        call_long_obs = observed_value_rho_long_call,
        error_percentage = ((rho_long_call - observed_value_rho_long_call)/(observed_value_rho_long_call))*100
    )
    push!(greek_dataframe,rho_row);

    # gamma -
    gamma_row = (
        greek = "gamma", 
        call_long_calc = gamma_long_call,
        call_short_calc = (sense)*gamma_long_call,
        check = gamma_long_call + (sense)*gamma_long_call,
        call_long_obs = observed_value_gamma_long_call,
        error_percentage = ((gamma_long_call - observed_value_gamma_long_call)/(observed_value_gamma_long_call))*100
    )
    push!(greek_dataframe,gamma_row);

    # display -
    greek_dataframe
end

and the `NVDA` put contract:

In [ ]:
let
    
    sense = -1;
    greek_dataframe = DataFrame();

    # delta -
    delta_row = (
        greek = "delta", 
        put_long_calc = delta_long_put,
        put_short_calc = (sense)*delta_long_put,
        check = delta_long_put + (sense)*delta_long_put,
        put_long_obs = observed_value_delta_long_put,
        error_percentage = ((delta_long_put - observed_value_delta_long_put)/(observed_value_delta_long_put))*100
    )
    push!(greek_dataframe,delta_row);

    # theta
    theta_row = (
        greek = "theta", 
        put_long_calc = theta_long_put,
        put_short_calc = (sense)*theta_long_put,
        check = theta_long_put + (sense)*theta_long_put,
        put_long_obs = observed_value_theta_long_put,
        error_percentage = ((theta_long_put - observed_value_theta_long_put)/(observed_value_theta_long_put))*100,
    )
    push!(greek_dataframe,theta_row);

    # vega -
    vega_row = (
        greek = "vega", 
        put_long_calc = vega_long_put,
        put_short_calc = (sense)*vega_long_put,
        check = vega_long_put + (sense)*vega_long_put,
        put_long_obs = observed_value_vega_long_put,
        error_percentage = ((vega_long_put - observed_value_vega_long_put)/(observed_value_vega_long_put))*100
    )
    push!(greek_dataframe,vega_row);

    # rho -
    rho_row = (
        greek = "rho", 
        put_long_calc = rho_long_put,
        put_short_calc = (sense)*rho_long_put,
        check = rho_long_put + (sense)*rho_long_put,
        put_long_obs = observed_value_rho_long_put,
        error_percentage = ((rho_long_put - observed_value_rho_long_put)/(observed_value_rho_long_put))*100
    )
    push!(greek_dataframe,rho_row);

    # gamma -
    gamma_row = (
        greek = "gamma", 
        put_long_calc = gamma_long_put,
        put_short_calc = (sense)*gamma_long_put,
        check = gamma_long_put + (sense)*gamma_long_put,
        put_long_obs = observed_value_gamma_long_put,
        error_percentage = ((gamma_long_put - observed_value_gamma_long_put)/(observed_value_gamma_long_put))*100
    )
    push!(greek_dataframe,gamma_row);

    # display -
    greek_dataframe
end

__Summary__: In general, our estimates of [the Greeks](https://en.wikipedia.org/wiki/en:Greeks_(finance)?variant=zh-tw) have percentage error in the range of `1-5%` for both the call and put contracts. The exception is $\Gamma$, with a percentage error above `30%` in both cases (we overestimated the $\Gamma$ in each case). However, the $\Gamma$ value is small, and the sign was correctly captured, so while not ideal, the impact on the premium will be small.

## Disclaimer and Risks
__This content is offered solely for training and informational purposes__. No offer or solicitation to buy or sell securities or derivative products or any investment or trading advice or strategy is made, given, or endorsed by the teaching team. 

__Trading involves risk__. Carefully review your financial situation before investing in securities, futures contracts, options, or commodity interests. Past performance, whether actual or indicated by historical tests of strategies, is no guarantee of future performance or success. Trading is generally inappropriate for someone with limited resources, investment or trading experience, or a low-risk tolerance.  Only risk capital that is not required for living expenses.

__You are fully responsible for any investment or trading decisions you make__. You should decide solely based on your financial circumstances, investment or trading objectives, risk tolerance, and liquidity needs.